In [16]:
# import data
import os
res1=[]
res2=[]
count=0
g=os.walk("D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3")
for path,d,filelist in g:
    for filename in filelist:
        if filename.endswith("jpg"):
            count=count+1
            print(os.path.join(path,filename))
            if count%2==1:
                res1.append(os.path.join(path,filename))
            else:
                res2.append(os.path.join(path,filename))

D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90003\90003d13.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90003\90003d17.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90004\90004d13.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90004\90004d17.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90006\90006d14.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90006\90006d15.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90008\90008d23.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90008\90008d24.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90010\90010d40.jpg
D:\Users\Yunjie\Documents\courses\CS559 Biometrics\Assignment\Assignment3\33333\90010\90010d42.jpg
D:\Users\Y

In [18]:
##craete the table

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

names=[]
for i in range(len(res1)):
    names.append(res1[i][86:91])
result={i:[0]*145 for i in sorted(names)}
frame =pd.DataFrame(result,index=sorted(names))

In [20]:
## connect the server

API_KEY = "PVynCByIjizaERKxDS2kv6saqdTeGLne"
API_SECRET = "1WD6ozOX8IuqvOt7zLl1L5BZ2YJh_ay7"
api_server_international = 'https://api-us.faceplusplus.com/facepp/v3/'
from facepp import API, File
api = API(API_KEY, API_SECRET,srv=api_server_international)
length=len(res1)
res1_token={}
res2_token={}

In [21]:
#  store token and compare same person
for i in range(0,145):
    face_one=res1[i]
    face_two=res2[i]
    face_info1 = api.detect(image_file=File(face_one))
    face_info2 = api.detect(image_file=File(face_two))
    key1=res1[i][86:91]
    key2=res2[i][86:91]
    res1_token[key1]=face_info1["faces"][0]["face_token"]
    res2_token[key2]=face_info2["faces"][0]["face_token"]
    compare_result=api.compare(face_token1=res1_token[key1],face_token2=res2_token[key2])
    frame[key1][key2]=compare_result['confidence']


In [24]:
#define check function
def check(frame):
    count=0
    for i in frame.columns[0:]:
        if frame[i]["90330"] ==0:
            return count
        count=count+1
    return -1

In [25]:
# def comapre differnet person function 
writer=pd.ExcelWriter('output.xlsx')
def compare_diff(start):
    for i in sorted(res1_token.keys())[start:]:#change the start position
        for j in sorted(res2_token.keys()):
            if i !=j:
                compare_result=api.compare(face_token1=res1_token[i],face_token2=res2_token[j])
                frame[i][j]=compare_result['confidence'] 
        writer=pd.ExcelWriter('output.xlsx')
        frame.to_excel(writer,'Sheet1')
        writer.save()

In [26]:
#run comapre differnet person function
done=False
start=0
while(done != True):
    try:
        compare_diff(start)
        if check(frame)==-1:
            done=True
    except:
        start=check(frame)
        writer=pd.ExcelWriter('output.xlsx')
        frame.to_excel(writer,'Sheet1')
        writer.save()

caught error: ('The read operation timed out',); retrying
